# TESTE PRÁTICO DE LINGUAGEM DE CONSULTA ESTRUTURADA (SQL)
Para a realização das atividades, você deverá executar as instruções abaixo para importar as bibliotecas e funções necessárias para criar a base de dados.

Ao final, esperamos que você desenvolva os códigos selecionados e entregue o notebook com as consultas SQL desenvolvidas e o arquivo .db criado.

O teste importará três arquivos cruciais para o desenvolvimento dessa atividade. Todos estão no Githun (não mude as URL's).

*   usuarios.csv
*   produtos.csv
*   vendas.csv

## O teste foi desenvolvido para ser executado com o SQLite3 e no Google Colab (Ubuntu 18.04). Você deverá utilizar consultas no padrão ANSI. Os comandos necessários podem ser encontrados em:

https://www.sqlite.org/docs.html

https://www.tutorialspoint.com/sqlite/index.htm



In [1]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.master("local[*]").getOrCreate()


In [84]:
%%bash
mkdir bases_teste
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/produtos.csv -o bases_teste/produtos.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/vendas.csv -o bases_teste/vendas.csv
curl https://raw.githubusercontent.com/A3Data/bases_testes/main/bases_teste/usuarios.csv -o bases_teste/usuarios.csv

Couldn't find program: 'bash'


In [76]:
# Setup Spark Session and Import Pandas

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("AtividadeSQL").getOrCreate()

In [87]:
def cria_tabela(path, nome_tabela):
    df = spark.read.csv(path, inferSchema=True, header=True)
    df.createOrReplaceTempView(nome_tabela)
    return df

usuarios = cria_tabela("bases_teste/usuarios.csv", "usuarios")
produtos = cria_tabela("bases_teste/produtos.csv", "produtos")
vendas = cria_tabela("bases_teste/vendas.csv", "vendas")

In [52]:
def q(query, n=30):
    return spark.sql(query).show(n=n, truncate=False)

# PERGUNTA 1:

### Escreva um comando em SQL que retorne a quantidade de vendas dos estados de Minas Gerais e São Paulo por ano e mês

In [108]:
q("""
SELECT
    YEAR(data_compra) AS ANO, 
    MONTH(data_compra) AS MES, 
    SUM(quantidade) AS QUANTIDADE, 
    B.estado AS ESTADO 

FROM vendas A

INNER JOIN usuarios AS B 
    on A.cod_usuario = B.cod_usuario

WHERE B.estado IN ('Minas Gerais','São Paulo')

GROUP BY ANO, MES, ESTADO

ORDER BY ESTADO, MES, ANO
""")

+----+---+----------+------------+
|ANO |MES|QUANTIDADE|ESTADO      |
+----+---+----------+------------+
|2019|1  |20        |Minas Gerais|
|2020|1  |8         |Minas Gerais|
|2021|1  |21        |Minas Gerais|
|2019|2  |2         |Minas Gerais|
|2020|2  |10        |Minas Gerais|
|2021|2  |75        |Minas Gerais|
|2020|3  |14        |Minas Gerais|
|2021|3  |6         |Minas Gerais|
|2019|4  |26        |Minas Gerais|
|2020|4  |1         |Minas Gerais|
|2021|4  |58        |Minas Gerais|
|2019|5  |11        |Minas Gerais|
|2020|5  |44        |Minas Gerais|
|2021|5  |22        |Minas Gerais|
|2018|6  |10        |Minas Gerais|
|2019|6  |1         |Minas Gerais|
|2020|6  |18        |Minas Gerais|
|2021|6  |16        |Minas Gerais|
|2019|7  |14        |Minas Gerais|
|2020|7  |30        |Minas Gerais|
|2021|7  |61        |Minas Gerais|
|2018|8  |16        |Minas Gerais|
|2019|8  |15        |Minas Gerais|
|2019|9  |27        |Minas Gerais|
|2020|9  |44        |Minas Gerais|
|2018|10 |20        

# PERGUNTA 2:

### Escreva um comando em SQL que retorne o ticket médio (valor médio de compra) e quantidade de compras realizadas dos usuários que fizeram compras entre 2019 e 2020 e que tiveram mais de 3 compras no período (lembre-se de contar todas as compras dos usuários que estão nessa condição!)

In [106]:
q(""" 

    SELECT 

        cod_usuario AS USUARIO,
        ROUND(AVG(valor), 2) AS TICKET_MEDIO, 
        COUNT(cod_usuario) AS QTD_DE_COMPRAS_REALIZADAS

    FROM vendas

    WHERE YEAR(data_compra) IN ('2019','2020')

    GROUP BY cod_usuario

    HAVING COUNT(cod_usuario) > 3
     
    ORDER BY cod_usuario


""")

+-------+------------+-------------------------+
|USUARIO|TICKET_MEDIO|QTD_DE_COMPRAS_REALIZADAS|
+-------+------------+-------------------------+
|6      |6564.84     |8                        |
|9      |26867.9     |5                        |
|11     |5129.02     |6                        |
|13     |20362.84    |4                        |
|20     |2556.7      |6                        |
|22     |8098.95     |6                        |
|23     |5665.09     |7                        |
|30     |3904.03     |7                        |
|32     |11297.5     |4                        |
|36     |2189.96     |4                        |
|37     |7266.34     |4                        |
|39     |19094.71    |11                       |
|41     |4926.2      |10                       |
|45     |5499.56     |6                        |
|46     |4245.5      |5                        |
|48     |19883.98    |8                        |
|51     |18303.82    |6                        |
|54     |14791.17   

# PERGUNTA 3:

### Escreva um comando em SQL que retorne quais são os usuários por estado que mais compraram em todo o período analisado e, o número de compras realizadas, a quantidade total de itens comprados e valor total pago

In [105]:
q(""" 

    SELECT 

        v.cod_usuario AS USUARIO,
        u.estado AS ESTADO,
        COUNT(v.cod_usuario) AS NUMERO_DE_COMPRAS_REALIZADAS,
        SUM(quantidade) AS QTD_TOTAL_ITENS_COMPRADOS,
        ROUND(SUM(valor),2) AS VALOR_TOTAL_PAGO

    FROM vendas AS v

    INNER JOIN usuarios AS u on v.cod_usuario == u.cod_usuario

    GROUP BY USUARIO, ESTADO

    ORDER BY NUMERO_DE_COMPRAS_REALIZADAS DESC
    
""")

+-------+-------------------+----------------------------+-------------------------+----------------+
|USUARIO|ESTADO             |NUMERO_DE_COMPRAS_REALIZADAS|QTD_TOTAL_ITENS_COMPRADOS|VALOR_TOTAL_PAGO|
+-------+-------------------+----------------------------+-------------------------+----------------+
|2811   |Maranhão           |13                          |117                      |184101.89       |
|3403   |Rio Grande do Norte|10                          |94                       |136347.01       |
|2843   |Paraná             |10                          |114                      |137228.01       |
|2340   |Mato Grosso        |9                           |112                      |165197.3        |
|1074   |Santa Catarina     |9                           |66                       |159165.18       |
|2981   |Bahia              |9                           |70                       |126282.4        |
|28     |Rio Grande do Sul  |9                           |76                      

# PERGUNTA 4:

### Escreva um comando em SQL que retorne quais são os 3 produtos mais comprados dos estados da região Sul e Sudeste, a quantidade de itens comprados, o valor total pago e a média de preço paga

In [109]:
q(""" 

    SELECT 
        
        p.nome_produto AS NOME_PRODUTO,
        SUM(v.quantidade) AS QTD_ITENS_COMPRADOS,
        ROUND(SUM(v.valor), 2) AS VALOR_TOTAL_PAGO,
        ROUND(AVG(v.valor), 2) AS MEDIA_PRECO_PAGO

    FROM vendas as v

    INNER JOIN produtos AS p ON v.cod_produto == p.cod_produto
    LEFT JOIN usuarios AS u ON v.cod_usuario == u.cod_usuario

    WHERE u.estado IN ('Rio de Janeiro','Minas Gerais','São Paulo','Espirito Santo','Rio Grande do Sul', 'Paraná','Santa Catarina')

    GROUP BY p.nome_produto
    
    ORDER BY QTD_ITENS_COMPRADOS DESC

    LIMIT 3
""")

+---------------------------+-------------------+----------------+----------------+
|NOME_PRODUTO               |QTD_ITENS_COMPRADOS|VALOR_TOTAL_PAGO|MEDIA_PRECO_PAGO|
+---------------------------+-------------------+----------------+----------------+
|Alcool em Gel 70% Johnson  |280                |1397.2          |53.74           |
|Escrivaninha em L          |141                |61617.0         |4739.77         |
|Notebook Asus Intel Core i7|134                |615180.6        |43941.47        |
+---------------------------+-------------------+----------------+----------------+



# PERGUNTA 5:

### Escreva um comando em SQL que retorne o ticket médio e o número total de usuários que fizeram pelo menos uma compra por faixa etária

In [118]:
q(""" 

    SELECT 
        
        ROUND(AVG(v.valor), 2) AS TICKET_MEDIO,
        COUNT(v.cod_usuario) AS NUMERO_TOTAL_USUARIOS,  
        u.faixa_etaria AS FAIXA_ETARIA 

    FROM vendas AS v

    INNER JOIN usuarios AS u ON v.cod_usuario == u.cod_usuario
    
    GROUP BY FAIXA_ETARIA

""")

+------------+---------------------+------------------+
|TICKET_MEDIO|NUMERO_TOTAL_USUARIOS|FAIXA_ETARIA      |
+------------+---------------------+------------------+
|12203.02    |64                   |Entre 22 a 27 anos|
|8890.13     |43                   |Entre 10 a 15 anos|
|12725.95    |50                   |Entre 16 a 21 anos|
|10849.48    |80                   |Entre 50 a 61 anos|
|16727.19    |42                   |Entre 37 a 49 anos|
|10012.63    |39                   |Entre 62 a 70 anos|
|8233.04     |56                   |Entre 28 a 36 anos|
|11303.1     |29                   |Mais de 70 anos   |
+------------+---------------------+------------------+



# PERGUNTA 6:

### Escreva um comando em SQL que aplique um desconto de 10% em todas as vendas dos usuários que fizeram pelo menos 3 compras de produtos na mesma categoria, a partir da 4ª compra realizada. Exiba apenas os usuários que terão o desconto aplicado, mantendo todas as compras, o valor original e o valor com o desconto aplicado

In [206]:
teste = spark.sql("""

    SELECT

        cod_usuario,
        data_compra,
        categoria_produto,
        ROW_NUMBER() OVER(PARTITION BY cod_usuario, categoria_produto ORDER BY data_compra) as Row

    FROM vendas AS v

    INNER JOIN produtos AS p ON p.cod_produto = v.cod_produto

    GROUP BY cod_usuario, data_compra, categoria_produto

    ORDER BY cod_usuario, categoria_produto, data_compra

    
""")

tabela_temporaria = teste.createOrReplaceTempView('tabela_temporaria')

In [234]:
q("""

    SELECT
    
        * from tabela_temporaria

    where cod_usuario = 16

""")  

+-----------+-------------------+----------------------+---+
|cod_usuario|data_compra        |categoria_produto     |Row|
+-----------+-------------------+----------------------+---+
|16         |2021-01-18 00:00:00|Casa e bem-estar      |1  |
|16         |2021-04-29 00:00:00|Jogos                 |1  |
|16         |2021-05-09 00:00:00|Lazer                 |1  |
|16         |2020-11-20 00:00:00|Material de escritorio|1  |
|16         |2021-05-29 00:00:00|Produtos de limpeza   |1  |
|16         |2021-07-25 00:00:00|Produtos de limpeza   |2  |
|16         |2020-12-06 00:00:00|Tecnologia            |1  |
|16         |2020-12-11 00:00:00|Tecnologia            |2  |
|16         |2021-01-13 00:00:00|Tecnologia            |3  |
|16         |2021-02-16 00:00:00|Tecnologia            |4  |
|16         |2021-05-23 00:00:00|Tecnologia            |5  |
+-----------+-------------------+----------------------+---+



In [241]:
q("""

    SELECT
    
        v.cod_usuario AS USUARIOS,
        v.valor AS VALOR_ORIGINAL,
        CASE WHEN temp.Row > 3 THEN (v.valor*0.9) ELSE v.valor END AS VALOR_COM_DESCONTO,
        MAX(temp.Row) as MAXIMO

    FROM vendas AS v

    INNER JOIN produtos AS p ON v.cod_produto == p.cod_produto

    JOIN tabela_temporaria AS temp ON v.cod_usuario == temp.cod_usuario and p.categoria_produto == temp.categoria_produto

    WHERE V.cod_usuario = 16

    GROUP BY v.cod_usuario, v.valor, temp.Row, p.categoria_produto

    HAVING MAXIMO > 3

    ORDER BY v.cod_usuario, MAXIMO

""")  

+--------+--------------+------------------+------+
|USUARIOS|VALOR_ORIGINAL|VALOR_COM_DESCONTO|MAXIMO|
+--------+--------------+------------------+------+
|16      |32136.3       |28922.67          |4     |
|16      |4672.8        |4205.52           |4     |
|16      |26643.5       |23979.15          |4     |
|16      |68863.5       |61977.15          |4     |
|16      |41318.1       |37186.29          |4     |
|16      |4672.8        |4205.52           |5     |
|16      |32136.3       |28922.67          |5     |
|16      |68863.5       |61977.15          |5     |
|16      |26643.5       |23979.15          |5     |
|16      |41318.1       |37186.29          |5     |
+--------+--------------+------------------+------+



In [232]:
q("""

    SELECT
    
        v.cod_usuario AS USUARIOS,
        v.valor AS VALOR_ORIGINAL,
        CASE WHEN temp.Row > 3 THEN (v.valor*0.9) ELSE v.valor END AS VALOR_COM_DESCONTO,
        MAX(temp.Row) as MAXIMO

    FROM vendas AS v

    INNER JOIN produtos AS p ON v.cod_produto == p.cod_produto

    JOIN tabela_temporaria AS temp ON v.cod_usuario == temp.cod_usuario and p.categoria_produto == temp.categoria_produto and v.data_compra == temp.data_compra

    GROUP BY v.cod_usuario, v.valor, temp.Row, p.categoria_produto
    
    HAVING MAXIMO > 3

    ORDER BY v.cod_usuario

""")  

+--------+--------------+------------------+------+
|USUARIOS|VALOR_ORIGINAL|VALOR_COM_DESCONTO|MAXIMO|
+--------+--------------+------------------+------+
|3       |36727.2       |33054.479999999996|4     |
|6       |439.14        |395.226           |4     |
|16      |41318.1       |37186.29          |4     |
|16      |4672.8        |4205.52           |5     |
|17      |94.8          |85.32             |4     |
|17      |118.5         |106.65            |5     |
|23      |24709.31      |22238.379         |4     |
|34      |2699.7        |2429.73           |4     |
|37      |899.9         |809.91            |4     |
|39      |50499.9       |45449.91          |4     |
|41      |39.92         |35.928000000000004|4     |
|87      |19507.35      |17556.614999999998|4     |
|101     |7.9           |7.11              |4     |
|106     |6555.0        |5899.5            |4     |
|123     |10.0          |9.0               |4     |
|132     |16198.2       |14578.380000000001|4     |
|132     |99

# PERGUNTA 7

### Escreva um comando em SQL que retorna o produto, quantos usuários fizeram pelo menos uma compra desse produto e qual é o valor mínimo e máximo pago

In [155]:
q("""

    SELECT
    
        p.nome_produto AS PRODUTO, 
        COUNT(DISTINCT v.cod_usuario) AS QTD_USUARIOS,
        MIN(v.valor) AS VALOR_MINIMO_PAGO,
        MAX(v.valor) AS VALOR_MAXIMO_PAGO

    FROM vendas AS v

    INNER JOIN produtos AS p ON v.cod_produto == p.cod_produto
    LEFT JOIN usuarios AS u ON v.cod_usuario == u.cod_usuario

    GROUP BY PRODUTO
    
""")  

+-----------------------------------+------------+-----------------+-----------------+
|PRODUTO                            |QTD_USUARIOS|VALOR_MINIMO_PAGO|VALOR_MAXIMO_PAGO|
+-----------------------------------+------------+-----------------+-----------------+
|Bicicleta Caloi Aro 29             |775         |1345.78          |26915.6          |
|Cafeteira Nespresso                |832         |219.57           |4391.4           |
|Notebook Asus Intel Core i7        |825         |4590.9           |91818.0          |
|Aparelho de Barbear OneBlade Philco|818         |129.9            |2598.0           |
|SPA Intel 700 L                    |839         |5999.1           |119982.0         |
|Jogo Mortal Kombat 11 PS4          |847         |49.9             |998.0            |
|Umidificador de Ar Cadence         |788         |77.5             |1550.0           |
|Blusa Lacoste                      |812         |138.99           |2779.8           |
|Alcool em Gel 70% Johnson          |837   

# PERGUNTA 8

### Escreva um comando em SQL que retorne o total de compras realizadas, o total de itens comprados e a receita total obtida

In [149]:
q("""

    SELECT
    
        COUNT(cod_usuario) AS TOTAL_COMPRAS_REALIZADAS,
        SUM(quantidade) AS TOTAL_ITENS_COMPRADOS,
        CAST(SUM(valor) AS NUMERIC) AS RECEITA_TOTAL_OBTIDA

    FROM vendas
    
""")  

+------------------------+---------------------+--------------------+
|TOTAL_COMPRAS_REALIZADAS|TOTAL_ITENS_COMPRADOS|RECEITA_TOTAL_OBTIDA|
+------------------------+---------------------+--------------------+
|20000                   |209149               |215849002           |
+------------------------+---------------------+--------------------+



# PERGUNTA 9

### Escreva um comando em SQL que retorne o produto, a quantidade de itens vendidos e a receita do produto que tem mais itens vendidos por ano e mês

In [142]:
q("""

    SELECT
        p.nome_produto AS NOME_PRODUTO,
        SUM(v.quantidade) AS QTD_ITENS_VENDIDOS,
        ROUND(SUM(v.valor), 2) AS RECEITA_PRODUTO,
        YEAR(data_compra) AS ANO,
        MONTH(data_compra) AS MES

    FROM vendas AS v

    INNER JOIN produtos AS p ON v.cod_produto == p.cod_produto

    GROUP BY NOME_PRODUTO, ANO, MES
   
    ORDER BY QTD_ITENS_VENDIDOS DESC

""")  

+-----------------------------------+------------------+---------------+----+---+
|NOME_PRODUTO                       |QTD_ITENS_VENDIDOS|RECEITA_PRODUTO|ANO |MES|
+-----------------------------------+------------------+---------------+----+---+
|SPA Intel 700 L                    |1628              |9766534.8      |2021|7  |
|Jogo Mortal Kombat 11 PS4          |1442              |71955.8        |2021|7  |
|Mochila Targus                     |1420              |83638.0        |2021|7  |
|Monitor LG 19 pol curvado          |1365              |797296.5       |2021|7  |
|Alcool em Gel 70% Johnson          |1361              |6791.39        |2021|7  |
|Cafeteira Nespresso                |1336              |293345.52      |2021|7  |
|Escrivaninha em L                  |1335              |583395.0       |2021|7  |
|Lenço umedecido Turma da Monica    |1322              |10443.8        |2021|7  |
|Kit 3 barras de chocolate Alpino   |1296              |12830.4        |2021|7  |
|Bicicleta Caloi

# PERGUNTA 10

### Escreva um comando em SQL que exiba quem são os usuários que compraram produtos diferentes

In [141]:
q("""

    SELECT
        cod_usuario AS USUARIOS

    FROM vendas

    GROUP BY USUARIOS

    HAVING COUNT(DISTINCT cod_produto) > 1

    ORDER BY USUARIOS


""")    

+--------+
|USUARIOS|
+--------+
|1       |
|2       |
|3       |
|4       |
|5       |
|6       |
|7       |
|8       |
|9       |
|10      |
|11      |
|12      |
|13      |
|14      |
|15      |
|16      |
|17      |
|18      |
|19      |
|20      |
|21      |
|22      |
|23      |
|24      |
|25      |
|26      |
|27      |
|28      |
|29      |
|30      |
+--------+
only showing top 30 rows



# PERGUNTA 11

### Escreva um comando em SQL que verifique usuários que não fizeram nenhuma compra

In [114]:
q("""

    SELECT
        cod_usuario AS USUARIOS 
    
    FROM usuarios

    WHERE cod_usuario NOT IN
        (SELECT a.cod_usuario FROM vendas a)

ORDER BY cod_usuario

""")



+--------+
|USUARIOS|
+--------+
|3499    |
|3504    |
|3516    |
|3556    |
|3560    |
|3564    |
|3692    |
|3754    |
|3764    |
|3829    |
|3837    |
|3861    |
|3879    |
|4020    |
|4251    |
|4269    |
|4334    |
|4391    |
|4431    |
|4443    |
|4725    |
|4850    |
|4909    |
|4995    |
|5049    |
|5098    |
|5133    |
|5229    |
|5245    |
|5247    |
+--------+
only showing top 30 rows

